In [ ]:
# Relevant imports
import torch_geometric
import networkx as nx
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np
from torch_geometric.data import Data
import torch
import os
from datageneration import create_conflict_dataset, create_conflict_dataset_irregular
from plotly.colors import sample_colorscale
from torch_geometric.utils import to_undirected, coalesce, degree as pyg_degree
from torch_geometric.utils import to_networkx
from torch_geometric.loader import DataLoader

In [2]:
# set random seed
random_seed = 42
np.random.seed(random_seed)
torch.manual_seed(random_seed)

In [3]:

def plot_using_plotly(
        G: nx.Graph,
        name_of_plot: str = "Network graph",
        *,
        spread: float = 1.6,       
        layout: str = "spring",    
        colourscale: str = "YlGnBu"
    ):
    n = G.number_of_nodes()
    if layout == "spring":
        k_val = spread / np.sqrt(n)   
        pos = nx.spring_layout(G, k=k_val, iterations=200, seed=42)
    elif layout == "kamada_kawai":
        pos = nx.kamada_kawai_layout(G)
        # amplify distances a bit for small graphs
        pos = {v: spread * np.array(p) for v, p in pos.items()}
    else:
        raise ValueError("layout must be 'spring' or 'kamada_kawai'")

    # centre & rescale to roughly [-1,1]² so long edges aren’t cropped
    xy = np.array(list(pos.values()))
    xy = xy - xy.mean(0)
    rng = xy.ptp()
    if rng > 0:
        xy = xy / rng * 2.0
    pos = {v: xy[i] for i, v in enumerate(pos)}

    edge_x, edge_y = [], []
    for u, v in G.edges():
        x0, y0 = pos[u]
        x1, y1 = pos[v]
        edge_x += [x0, x1, None]
        edge_y += [y0, y1, None]

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        mode="lines",
        hoverinfo="none",
        line=dict(width=1, color="rgba(120,120,120,0.45)")
    )

    deg_dict      = dict(G.degree())
    unique_deg    = sorted(set(deg_dict.values()))
    idx_of_degree = {d: i for i, d in enumerate(unique_deg)}

    # build a discrete palette by sampling the continuous scale
    palette = sample_colorscale(colourscale, [i / max(1, len(unique_deg)-1)
                                              for i in range(len(unique_deg))])

    node_x  = [pos[v][0] for v in G.nodes()]
    node_y  = [pos[v][1] for v in G.nodes()]
    node_sz = [6 + 3 * deg_dict[v] for v in G.nodes()]
    node_clr_idx = [idx_of_degree[deg_dict[v]] for v in G.nodes()]
    node_clr = [palette[i] for i in node_clr_idx]
    node_txt = [f'#{v}: {deg_dict[v]} links' for v in G.nodes()]

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode="markers",
        hoverinfo="text",
        text=node_txt,
        marker=dict(
            size=node_sz,
            color=node_clr,
            line_width=1,
            showscale=True,
            colorscale=palette,  
            cmin=0,
            cmax=len(palette) - 1,
            colorbar=dict(
                title="Degree",
                tickvals=list(range(len(unique_deg))),
                ticktext=[str(d) for d in unique_deg]
            )
        )
    )

    fig = go.Figure(
        data=[edge_trace, node_trace],
        layout=go.Layout(
            title=name_of_plot,
            hovermode="closest",
            showlegend=False,
            margin=dict(l=5, r=5, t=40, b=20),
            xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        )
    )
    fig.show()


### Create and analyze regular dataset 

In [4]:
num_samples = {"train": 10, "test": 50}
min_n = 200
max_n = 300
# set random seed
random_seed = 32
deltavalue = 0.05
target_degree = 6 # which means the original degree is 4
r = 1.2
np.random.seed(random_seed)
torch.manual_seed(random_seed)
loader, data, data_test, orig_data, first_A, degrees = create_conflict_dataset(num_samples, min_n, 
                                            max_n, r, deltavalue, dual_tol=0.5, batch_size=1, 
                                            target_K=500, target_dual_degree=target_degree, 
                                            noise = 0, 
                                            return_data_for_plotting=True)

orig edges 1072
K 536
D 5.5895524
1
orig edges 750
K 375
orig edges 934
K 467
D 5.2591004
orig edges 1040
K 520
D 5.230769
orig edges 840
K 420
orig edges 678
K 339
orig edges 1012


/home/alelab/miniconda3/envs/sawl2/lib/python3.8/site-packages/torch_geometric/data/storage.py:303: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'edge_index'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


K 506
D 5.146245
orig edges 824
K 412
orig edges 938
K 469
D 5.552239
2
orig edges 866
K 433
orig edges 966
K 483
D 5.2919254
orig edges 978
K 489
D 5.3333335
orig edges 980
K 490
D 5.053061
orig edges 1030
K 515
D 5.491262
orig edges 1040
K 520
D 5.496154
orig edges 1034
K 517
D 5.187621
orig edges 774
K 387
orig edges 798
K 399
orig edges 1034
K 517
D 5.2108316
orig edges 952
K 476
D 5.5882354
3
orig edges 1008
K 504
D 5.452381
orig edges 960
K 480
D 5.616667
4
orig edges 1040
K 520
D 5.503846
5
orig edges 724
K 362
orig edges 898
K 449
orig edges 764
K 382
orig edges 974
K 487
D 5.3511295
orig edges 700
K 350
orig edges 882
K 441
orig edges 674
K 337
orig edges 760
K 380
orig edges 934
K 467
D 5.17773
orig edges 826
K 413
orig edges 712
K 356
orig edges 920
K 460
D 5.1826086
orig edges 754
K 377
orig edges 856
K 428
orig edges 1012
K 506
D 5.458498
orig edges 1034
K 517
D 5.489362
orig edges 1026
K 513
D 5.45809
orig edges 1008
K 504
D 5.4325395
orig edges 840
K 420
orig edges 782
K

In [5]:
max_deg = 30
degree_counts = np.zeros((len(degrees), max_deg + 1))

for i, degrees_i in enumerate(degrees):
    hist = np.bincount(degrees_i, minlength=max_deg + 1)
    degree_counts[i, :] = hist

mean_counts = degree_counts.mean(axis=0)

plt.figure(figsize=(8, 5))
plt.bar(np.arange(max_deg + 1), mean_counts, color='steelblue', edgecolor='black')
plt.xlabel('Degree')
plt.ylabel('Average number of nodes')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.xticks(np.arange(max_deg + 1))
plt.tight_layout()
os.makedirs('plots', exist_ok=True)
image_path = os.path.join('plots', 'degrees_dual_graph_{}.png'.format(target_degree))
plt.savefig(image_path)
plt.close()

deg_orig_all = []      
for g in orig_data:
    # make edges undirected & remove duplicate directions
    ei = coalesce(to_undirected(g.edge_index, num_nodes=g.num_nodes))
    deg = pyg_degree(ei[0], g.num_nodes).to(torch.long)
    deg_orig_all.append(deg)

max_deg_orig = max(int(d.max()) for d in deg_orig_all)
deg_hist = np.zeros((len(deg_orig_all), max_deg_orig + 1))

for i, deg in enumerate(deg_orig_all):
    hist = np.bincount(deg.numpy(), minlength=max_deg_orig + 1)
    deg_hist[i, :] = hist         

mean_counts_orig = deg_hist.mean(axis=0)

plt.figure(figsize=(8, 5))
plt.bar(np.arange(max_deg_orig + 1), mean_counts_orig,
        color='indianred', edgecolor='black')

plt.xlabel('Degree')
plt.ylabel('Average number of nodes')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.xticks(np.arange(max_deg_orig + 1))

plt.tight_layout()
os.makedirs('plots', exist_ok=True)
plt.savefig('plots/degrees_comm_graph_{}.png'.format(int((target_degree+2)/2)))
plt.close()


In [6]:
Ks = []
num_edges = []
for sample in range(num_samples['train']):
    Ks.append(data[sample].A.shape)
    num_edges.append(len(data[sample].edge_index[0])/2)
for sample in range(num_samples['test']):
    Ks.append(data_test[sample].A.shape)
    num_edges.append(len(data_test[sample].edge_index[0])/2)
print("K_min = {}, K_mean = {}, K_max = {}".format(np.min(np.array(Ks)), np.mean(np.array(Ks)), np.max(np.array(Ks))))
print("num_edges_min = {}, num_edges_mean = {}, num_edges_max = {}".format(np.min(np.array(num_edges)), np.mean(np.array(num_edges)), np.max(np.array(num_edges))))

K_min = 457, K_mean = 498.21666666666664, K_max = 544
num_edges_min = 1257.0, num_edges_mean = 1385.4333333333334, num_edges_max = 1534.0


In [ ]:
dataset_file  = 'sawl_dataset_degree6_graphs_60.pt'
print("dataset →", dataset_file)

data = torch.load(os.path.join("data", dataset_file))
train_data, test_data = data["train"], data["test"]
data = train_data
for sample in range(10):
        graph = Data(x=data[sample].x, edge_index=data[sample].edge_index)
        G = torch_geometric.utils.to_networkx(graph, to_undirected=True)
        plot_using_plotly(G, name_of_plot='Conflict graph')

In [8]:
dataset_file  = 'sawl_dataset_degree{}_graphs_60.pt'.format(target_degree)
print("dataset ", dataset_file)
data = torch.load(os.path.join("data", dataset_file))
train_data, test_data = data["train"], data["test"]

loader = {"train": DataLoader(train_data, batch_size=1, shuffle=True), "test" : DataLoader(test_data , batch_size=1)}

objs_IS, cons_IS, num_edges, Ks = [], [], [], []

phase = "test"       
how_many = 50  
mis_sizes = []           

for idx, gd in enumerate(data[phase][:how_many]):
    K = gd.K
    G = nx.Graph(to_networkx(gd, to_undirected=True)) 
    IS = nx.approximation.maximum_independent_set(G)
    mis_size = len(IS)
    mis_sizes.append(mis_size)

    print(f"[{phase} #{idx}]  K = {K:3d} ,  |MIS| = {mis_size:3d}  "
          f"({mis_size / K:.2%} of K)")

mis_sizes = np.array(mis_sizes)
print("\n==  Independent-set size summary  ==")
print(f"min = {mis_sizes.min():3d}   "
      f"mean = {mis_sizes.mean():6.2f}   "
      f"max = {mis_sizes.max():3d}")

dataset  sawl_dataset_degree6_graphs_60.pt
[test #0]  K = 476 ,  |MIS| = 117  (24.58% of K)
[test #1]  K = 533 ,  |MIS| = 131  (24.58% of K)
[test #2]  K = 518 ,  |MIS| = 126  (24.32% of K)
[test #3]  K = 477 ,  |MIS| = 115  (24.11% of K)
[test #4]  K = 516 ,  |MIS| = 126  (24.42% of K)
[test #5]  K = 525 ,  |MIS| = 132  (25.14% of K)
[test #6]  K = 465 ,  |MIS| = 113  (24.30% of K)
[test #7]  K = 465 ,  |MIS| = 115  (24.73% of K)
[test #8]  K = 480 ,  |MIS| = 119  (24.79% of K)
[test #9]  K = 476 ,  |MIS| = 118  (24.79% of K)
[test #10]  K = 480 ,  |MIS| = 118  (24.58% of K)
[test #11]  K = 533 ,  |MIS| = 131  (24.58% of K)
[test #12]  K = 476 ,  |MIS| = 115  (24.16% of K)
[test #13]  K = 525 ,  |MIS| = 130  (24.76% of K)
[test #14]  K = 465 ,  |MIS| = 113  (24.30% of K)
[test #15]  K = 472 ,  |MIS| = 113  (23.94% of K)
[test #16]  K = 517 ,  |MIS| = 126  (24.37% of K)
[test #17]  K = 480 ,  |MIS| = 117  (24.38% of K)
[test #18]  K = 461 ,  |MIS| = 115  (24.95% of K)
[test #19]  K = 5

### Create and analyze irregular dataset

In [ ]:
num_samples = {"train": 100, "test": 100}
nodes = [15, 16, 17, 18, 19]
# set random seed again just in case
random_seed = 32
deltavalue = 0.05
target_degree = 6 # which means the original degree is 4
r = 1.2
noise = 0.01 # 0.01 used for everything except to test different lambda distributions
np.random.seed(random_seed)
torch.manual_seed(random_seed)
density = False
density_level = 0
loader, data, data_test, orig_data, first_A, degrees = create_conflict_dataset_irregular(num_samples, nodes,
                                            r, deltavalue, batch_size=1, 
                                            target_K=500, target_dual_degree=target_degree, 
                                            noise = noise, dual_tol=0.5,
                                            density=density, density_level=density_level,
                                            return_data_for_plotting=True)


In [ ]:
max_deg = 12
degree_counts = np.zeros((len(degrees), max_deg + 1))
plt.rcParams.update({'font.size': 20})

for i, degrees_i in enumerate(degrees):
    hist = np.bincount(degrees_i, minlength=max_deg + 1)
    degree_counts[i, :] = hist

mean_counts = degree_counts.mean(axis=0)

plt.figure(figsize=(8, 5))
plt.bar(np.arange(max_deg + 1), mean_counts, color='steelblue', edgecolor='black')
plt.xlabel('Degree')
plt.ylabel('Average number of nodes')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.xticks(np.arange(max_deg + 1))
plt.tight_layout()
os.makedirs('plots', exist_ok=True)
image_path = os.path.join('plots', 'conflict_noise{}.png'.format(noise))
plt.savefig(image_path)
plt.close()

deg_orig_all = []      
for g in orig_data:
    ei = coalesce(to_undirected(g.edge_index, num_nodes=g.num_nodes))
    deg = pyg_degree(ei[0], g.num_nodes).to(torch.long)
    deg_orig_all.append(deg)

max_deg_orig = max(int(d.max()) for d in deg_orig_all)
deg_hist = np.zeros((len(deg_orig_all), max_deg_orig + 1))

for i, deg in enumerate(deg_orig_all):
    hist = np.bincount(deg.numpy(), minlength=max_deg_orig + 1)
    deg_hist[i, :] = hist         

mean_counts_orig = deg_hist.mean(axis=0)

plt.figure(figsize=(8, 5))
plt.bar(np.arange(max_deg_orig + 1), mean_counts_orig,
        color='indianred', edgecolor='black')

plt.xlabel('Degree')
plt.ylabel('Average number of nodes')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.xticks(np.arange(max_deg_orig + 1))

plt.tight_layout()
os.makedirs('plots', exist_ok=True)
plt.savefig('plots/comm_noise{}.png'.format(noise))
plt.close()


In [ ]:
Ks = []
num_edges = []
for sample in range(num_samples['train']):
    Ks.append(data[sample].A.shape)
    num_edges.append(len(data[sample].edge_index[0])/2)
for sample in range(num_samples['test']):
    Ks.append(data_test[sample].A.shape)
    num_edges.append(len(data_test[sample].edge_index[0])/2)
print("K_min = {}, K_mean = {}, K_max = {}".format(np.min(np.array(Ks)), np.mean(np.array(Ks)), np.max(np.array(Ks))))
print("num_edges_min = {}, num_edges_mean = {}, num_edges_max = {}".format(np.min(np.array(num_edges)), np.mean(np.array(num_edges)), np.max(np.array(num_edges))))